# | default_exp Testing Polars for EDA, migrating cleaned data to Vespa

In [ ]:
# | hide
import adbc_driver_postgresql.dbapi
from datetime import datetime
from enum import Enum
from functools import partial
import json
import polars as pl
import requests
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import vespa

from src.backend.raw_data_cleaning import text_processing

In [ ]:
all_recipes_json_path = "../data/recipes-en-201706/allrecipes-recipes.json"

ar_df = pl.read_ndjson(all_recipes_json_path)

In [ ]:
ar_df.head(3)

author,cook_time_minutes,description,error,footnotes,ingredients,instructions,photo_url,prep_time_minutes,rating_stars,review_count,time_scraped,title,total_time_minutes,url
str,i64,str,bool,list[str],list[str],list[str],str,i64,f64,i64,i64,str,i64,str
"""Stephanie""",25,"""I just started adding my favor…",false,[],"[""1/2 cup unsalted butter, chilled and cubed"", ""1 cup chopped onion"", … ""1/2 cup chopped fresh basil""]","[""Preheat oven to 400 degrees F (205 degrees C). Butter a 9x9x2 inch baking pan."", ""Melt 1 tablespoon butter in medium nonstick skillet over medium-low heat. Add onion and saute until tender, about 10 minutes. Cool."", … ""Bake cornbread until golden and tester inserted comes out clean, about 45 minutes. Cool 20 minutes in pan. Cut cornbread into squares.""]","""http://images.media-allrecipes…",55,4.32,46,1498204021,"""Basil, Roasted Peppers and Mon…",100,"""http://allrecipes.com/Recipe/6…"
"""Stephanie""",0,"""These are great as an appetize…",false,[],"[""1/2 cup Parmesan cheese"", ""3/4 teaspoon ground black pepper"", … ""1 egg white""]","[""Combine parmesan cheese, pepper and garlic powder. Unfold pastry sheets onto cutting board. Brush lightly with egg white; sprinkle each sheet with 1/4 of the cheese mixture. Lightly press into pastry, turn over; repeat. Cut each sheet into 12 (1-inch) strips; twist."", ""Place on ungreased cookie sheet and bake in 350 degrees F (175 degrees C) oven for 15 minutes or until golden brown.""]","""http://images.media-allrecipes…",0,4.18,44,1498204301,"""Crispy Cheese Twists""",0,"""http://allrecipes.com/Recipe/6…"
"""Kathy""",0,"""This is the best bread recipe.…",false,[],"[""2 cups hot water"", ""1/2 cup margarine"", … ""2 eggs""]","[""Melt margarine in hot water. Add sugar and salt and stir. Add cold water and yeast. Stir to dissolve yeast."", ""Add 3 cups flour and mix. Add eggs and 2 1/2 - 3 cups more flour. Mix, cover and let rise until dough doubles in size."", … ""Make walnut size balls of dough. Place about 2 inches apart in well-buttered 9 x 13 inch pan. Bake in a preheated 350 degrees F (175 degrees C) oven for 30-45 minutes. Brush top of rolls with margarine while hot.""]","""http://images.media-allrecipes…",0,3.65,168,1498206088,"""Mom's Yeast Rolls""",0,"""http://allrecipes.com/Recipe/6…"


In [ ]:
ar_df

author,cook_time_minutes,description,error,footnotes,ingredients,instructions,photo_url,prep_time_minutes,rating_stars,review_count,time_scraped,title,total_time_minutes,url
str,i64,str,bool,list[str],list[str],list[str],str,i64,f64,i64,i64,str,i64,str
"""Stephanie""",25,"""I just started adding my favor…",false,[],"[""1/2 cup unsalted butter, chilled and cubed"", ""1 cup chopped onion"", … ""1/2 cup chopped fresh basil""]","[""Preheat oven to 400 degrees F (205 degrees C). Butter a 9x9x2 inch baking pan."", ""Melt 1 tablespoon butter in medium nonstick skillet over medium-low heat. Add onion and saute until tender, about 10 minutes. Cool."", … ""Bake cornbread until golden and tester inserted comes out clean, about 45 minutes. Cool 20 minutes in pan. Cut cornbread into squares.""]","""http://images.media-allrecipes…",55,4.32,46,1498204021,"""Basil, Roasted Peppers and Mon…",100,"""http://allrecipes.com/Recipe/6…"
"""Stephanie""",0,"""These are great as an appetize…",false,[],"[""1/2 cup Parmesan cheese"", ""3/4 teaspoon ground black pepper"", … ""1 egg white""]","[""Combine parmesan cheese, pepper and garlic powder. Unfold pastry sheets onto cutting board. Brush lightly with egg white; sprinkle each sheet with 1/4 of the cheese mixture. Lightly press into pastry, turn over; repeat. Cut each sheet into 12 (1-inch) strips; twist."", ""Place on ungreased cookie sheet and bake in 350 degrees F (175 degrees C) oven for 15 minutes or until golden brown.""]","""http://images.media-allrecipes…",0,4.18,44,1498204301,"""Crispy Cheese Twists""",0,"""http://allrecipes.com/Recipe/6…"
"""Kathy""",0,"""This is the best bread recipe.…",false,[],"[""2 cups hot water"", ""1/2 cup margarine"", … ""2 eggs""]","[""Melt margarine in hot water. Add sugar and salt and stir. Add cold water and yeast. Stir to dissolve yeast."", ""Add 3 cups flour and mix. Add eggs and 2 1/2 - 3 cups more flour. Mix, cover and let rise until dough doubles in size."", … ""Make walnut size balls of dough. Place about 2 inches apart in well-buttered 9 x 13 inch pan. Bake in a preheated 350 degrees F (175 degrees C) oven for 30-45 minutes. Brush top of rolls with margarine while hot.""]","""http://images.media-allrecipes…",0,3.65,168,1498206088,"""Mom's Yeast Rolls""",0,"""http://allrecipes.com/Recipe/6…"
"""Mary E. Crain""",0,"""A Southern delight with a 'mor…",false,[],"[""1 1/2 cups white sugar"", ""1/2 cup vegetable oil"", … ""1/2 cup chopped pecans""]","[""Combine sugar and oil; beat well. Add eggs and beat. Combine flour, baking soda, salt, cinnamon and nutmeg. Stir flour mixture into egg mixture alternately with water. Stir in sweet potatoes and chopped nuts."", ""Pour batter into greased 9x5 inch loaf pan (or 2 small loaf pans). Bake at 350 degrees F (175 degrees C) for about one hour.""]","""http://images.media-allrecipes…",0,4.7,344,1498209193,"""Sweet Potato Bread I""",0,"""http://allrecipes.com/Recipe/6…"
"""DeeDee""",45,"""You can make these into braids…",false,"[""Cook's Note:"", ""I made 1 braid, 12 cinnamon buns, and 12 butterhorns with this recipe."", ""Parchment can be used for easier cleanup/removal from the pan.""]","[""1/4 cup butter"", ""1 teaspoon white sugar"", … ""1 teaspoon salt""]","[""Stir butter and 1 teaspoon sugar into the hot milk until butter is melted. When mixture is lukewarm, stir in yeast and set aside for 5 minutes."", ""When mixture is creamy, transfer to a large mixing bowl. Mix in 2 cups of bread flour. Add 1/2 cup sugar, eggs, orange juice, orange zest, and salt and beat until combined. Add remaining flour, mixing well after each addition, until it pulls away from the sides of the bowl."", … ""Bake until rolls or loaves are golden brown, 10 to 12 minutes for individual buns, about 20 minutes for braids. Frost rolls, if desired (see Cook's Note) or sprinkle with confectioners' sugar.""]","""http://images.media-allrecipes…",20,4.71,6,1498205251,"""Orange Buns""",170,"""http://allrecipes.com/Recipe/6…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
""

In [ ]:
# mealeon_id: use script
# language: english
# source_id: use URL
# title: use title
# origin: AllRecipes
# url
# ingredients
# photo_url
# description
# steps: use instructions
# cuisines

In [ ]:
ar_df = ar_df.with_columns(
    pl.concat_str(
        [
            pl.col("url"),
            pl.col("title").str.normalize("NFKC")
                            .str.replace_all(r" ", "-")
                            .str.replace_all(r"[^a-zA-Z0-9-]", "")
                            .str.to_lowercase()
        ],
        separator="",
    ).alias("working_url"),
    (pl.col("url").map_elements(partial(
                                    text_processing.mealeon_id_assigner, 
                                    "AllRecipes"
                                )
                                , return_dtype=pl.String)
                    .alias("mealeon_id")),
    (pl.col("instructions").alias("steps")),
    (pl.col("url").map_elements(text_processing.allrecipes_id_extractor, return_dtype=pl.String).alias("source_id")),
    (pl.lit("English").alias("language")),
    (pl.lit("AllRecipes").alias("origin")),
    (pl.lit([""]).alias("cuisines")),
).drop(
    ["url"]
).rename({"working_url":"url"})

ar_df.head(3)

author,cook_time_minutes,description,error,footnotes,ingredients,instructions,photo_url,prep_time_minutes,rating_stars,review_count,time_scraped,title,total_time_minutes,url,mealeon_id,steps,source_id,language,origin,cuisines
str,i64,str,bool,list[str],list[str],list[str],str,i64,f64,i64,i64,str,i64,str,str,list[str],str,str,str,list[str]
"""Stephanie""",25,"""I just started adding my favor…",false,[],"[""1/2 cup unsalted butter, chilled and cubed"", ""1 cup chopped onion"", … ""1/2 cup chopped fresh basil""]","[""Preheat oven to 400 degrees F (205 degrees C). Butter a 9x9x2 inch baking pan."", ""Melt 1 tablespoon butter in medium nonstick skillet over medium-low heat. Add onion and saute until tender, about 10 minutes. Cool."", … ""Bake cornbread until golden and tester inserted comes out clean, about 45 minutes. Cool 20 minutes in pan. Cut cornbread into squares.""]","""http://images.media-allrecipes…",55,4.32,46,1498204021,"""Basil, Roasted Peppers and Mon…",100,"""http://allrecipes.com/Recipe/6…","""AllRecipes-58c6dd19a6a89328e71…","[""Preheat oven to 400 degrees F (205 degrees C). Butter a 9x9x2 inch baking pan."", ""Melt 1 tablespoon butter in medium nonstick skillet over medium-low heat. Add onion and saute until tender, about 10 minutes. Cool."", … ""Bake cornbread until golden and tester inserted comes out clean, about 45 minutes. Cool 20 minutes in pan. Cut cornbread into squares.""]","""6664""","""English""","""AllRecipes""","[""""]"
"""Stephanie""",0,"""These are great as an appetize…",false,[],"[""1/2 cup Parmesan cheese"", ""3/4 teaspoon ground black pepper"", … ""1 egg white""]","[""Combine parmesan cheese, pepper and garlic powder. Unfold pastry sheets onto cutting board. Brush lightly with egg white; sprinkle each sheet with 1/4 of the cheese mixture. Lightly press into pastry, turn over; repeat. Cut each sheet into 12 (1-inch) strips; twist."", ""Place on ungreased cookie sheet and bake in 350 degrees F (175 degrees C) oven for 15 minutes or until golden brown.""]","""http://images.media-allrecipes…",0,4.18,44,1498204301,"""Crispy Cheese Twists""",0,"""http://allrecipes.com/Recipe/6…","""AllRecipes-59773cd5496dc7446c4…","[""Combine parmesan cheese, pepper and garlic powder. Unfold pastry sheets onto cutting board. Brush lightly with egg white; sprinkle each sheet with 1/4 of the cheese mixture. Lightly press into pastry, turn over; repeat. Cut each sheet into 12 (1-inch) strips; twist."", ""Place on ungreased cookie sheet and bake in 350 degrees F (175 degrees C) oven for 15 minutes or until golden brown.""]","""6663""","""English""","""AllRecipes""","[""""]"
"""Kathy""",0,"""This is the best bread recipe.…",false,[],"[""2 cups hot water"", ""1/2 cup margarine"", … ""2 eggs""]","[""Melt margarine in hot water. Add sugar and salt and stir. Add cold water and yeast. Stir to dissolve yeast."", ""Add 3 cups flour and mix. Add eggs and 2 1/2 - 3 cups more flour. Mix, cover and let rise until dough doubles in size."", … ""Make walnut size balls of dough. Place about 2 inches apart in well-buttered 9 x 13 inch pan. Bake in a preheated 350 degrees F (175 degrees C) oven for 30-45 minutes. Brush top of rolls with margarine while hot.""]","""http://images.media-allrecipes…",0,3.65,168,1498206088,"""Mom's Yeast Rolls""",0,"""http://allrecipes.com/Recipe/6…","""AllRecipes-4022751697d6c412320…","[""Melt margarine in hot water. Add sugar and salt and stir. Add cold water and yeast. Stir to dissolve yeast."", ""Add 3 cups flour and mix. Add eggs and 2 1/2 - 3 cups more flour. Mix, cover and let rise until dough doubles in size."", … ""Make walnut size balls of dough. Place about 2 inches apart in well-buttered 9 x 13 inch pan. Bake in a preheated 350 degrees F (175 degrees C) oven for 30-45 minutes. Brush top of rolls with margarine while hot.""]","""6665""","""English""","""AllRecipes""","[""""]"


In [ ]:
# ar_df.write_json("../data/recipes-en-201706/allrecipes-fixed-urls.json")

# only unique recipes
# ar_df.select(pl.count("url").len())
ar_df.shape


(225602, 21)

In [ ]:
ar_df.unique(subset=["url"])

author,cook_time_minutes,description,error,footnotes,ingredients,instructions,photo_url,prep_time_minutes,rating_stars,review_count,time_scraped,title,total_time_minutes,url,mealeon_id,steps,source_id,language,origin,cuisines
str,i64,str,bool,list[str],list[str],list[str],str,i64,f64,i64,i64,str,i64,str,str,list[str],str,str,str,list[str]
"""The Kitchen at Johnsonville Sa…",0,"""Squares of cheesy pizza are se…",false,[],"[""1 (12 inch) pre-baked pizza crust"", ""1 1/2 cups shredded mozzarella cheese"", … ""1 (3.5 ounce) package sliced pepperoni""]","[""Top pizza crust with cheese. Bake crust according to package directions. Meanwhile, in a 12-inch skillet, combine the pizza sauce, sausage and pepperoni. Bring to a simmer, stirring occasionally. Reduce heat to low. Cut pizza crust into squares; arrange over sausage mixture in skillet.""]","""http://images.media-allrecipes…",0,0.0,0,1498211728,"""Johnsonville® Three Cheese Ita…",30,"""http://allrecipes.com/Recipe/8…","""AllRecipes-4874db3ea8d0805e068…","[""Top pizza crust with cheese. Bake crust according to package directions. Meanwhile, in a 12-inch skillet, combine the pizza sauce, sausage and pepperoni. Bring to a simmer, stirring occasionally. Reduce heat to low. Cut pizza crust into squares; arrange over sausage mixture in skillet.""]","""83025""","""English""","""AllRecipes""","[""""]"
"""PJ's kitchen""",25,"""My mother used to bake these c…",false,[],"[""1 cup all-purpose flour"", ""1 teaspoon baking powder"", … ""3 cups whole pitted dates""]","[""Preheat oven to 375 degrees F (190 degrees C). Grease 2 baking sheets. In a bowl, sift together the flour, baking powder, and salt."", ""Beat butter and sugar in a bowl until light and fluffy with an electric mixer set on medium speed. Add egg and vanilla extract to mixture and beat in. Mix in the flour mixture in 3 additions, alternating with cream. Mixture should form a thick batter."", … ""Bake one sheet at a time in the center of the preheated oven until the batter is lightly browned, about 10 minutes.""]","""http://images.media-allrecipes…",20,3.0,2,1498154184,"""Mother's Date Dips from 1940's""",60,"""http://allrecipes.com/Recipe/2…","""AllRecipes-f63b1f8129a3fece9e1…","[""Preheat oven to 375 degrees F (190 degrees C). Grease 2 baking sheets. In a bowl, sift together the flour, baking powder, and salt."", ""Beat butter and sugar in a bowl until light and fluffy with an electric mixer set on medium speed. Add egg and vanilla extract to mixture and beat in. Mix in the flour mixture in 3 additions, alternating with cream. Mixture should form a thick batter."", … ""Bake one sheet at a time in the center of the preheated oven until the batter is lightly browned, about 10 minutes.""]","""216566""","""English""","""AllRecipes""","[""""]"
"""Reynolds Kitchens""",20,"""Tuck grilled salmon, lettuce, …",false,"[""REYNOLDS KITCHENS TIP:"", ""Celebrate the flavors of Greece with this delicious hand-held meal! Grilling the salmon in foil packets helps seal in all of the juices and infuses the fish with fresh lemon flavor  and keeps it from drying out. Plus cleanup is a breeze!""]","[""1 (1 pound) fresh or frozen skinless salmon fillet"", ""Reynolds Wrap® Heavy Duty Aluminum Foil"", … ""1 tablespoon snipped fresh dill weed""]","[""Thaw fish if frozen. Rinse and pat dry with paper towels. Preheat grill to medium-high."", ""Tear off an 18x18-inch sheet of Reynolds Wrap(R) Heavy Duty Aluminum Foil. Place onion in the center of the sheet. Top with salmon. Drizzle with the olive oil and sprinkle with the salt and black pepper. Top with lemon slices. Bring up foil sides. Double-fold top and ends to seal packet, leaving room for heat circulation inside."", … ""Assemble sandwiches by topping pita with the salmon, onions, lettuce, and tomatoes. Fold in half to serve. In a small bowl, mix together the yogurt, basil, and dill. Serve over pita sandwiches.""]","""http://images.media-allrecipes…",14,4.14,21,1498163263,"""Grilled Salmon Greek Pitas""

In [ ]:
# scrapy shell
# from scrapy import Request

# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:50.0) Gecko/20100101 Firefox/50.0'}

# target_url = "https://www.allrecipes.com/recipe/202261/brazilian-style-flan-pudim-de-leite-condensado/"

# req = Request(target_url, headers=headers)
# fetch(req)

# response.xpath('//*[@id="mntl-universal-breadcrumbs_1-0"]/li/a/span/text()').getall()

In [ ]:
reduced_ar_df = ar_df.select(["mealeon_id", "language", "source_id", "title", "origin", "url", "ingredients", "photo_url", "description", "steps", "cuisines"])
reduced_ar_df.head(3)

mealeon_id,language,source_id,title,origin,url,ingredients,photo_url,description,steps,cuisines
str,str,str,str,str,str,list[str],str,str,list[str],list[str]
"""AllRecipes-58c6dd19a6a89328e71…","""English""","""6664""","""Basil, Roasted Peppers and Mon…","""AllRecipes""","""http://allrecipes.com/Recipe/6…","[""1/2 cup unsalted butter, chilled and cubed"", ""1 cup chopped onion"", … ""1/2 cup chopped fresh basil""]","""http://images.media-allrecipes…","""I just started adding my favor…","[""Preheat oven to 400 degrees F (205 degrees C). Butter a 9x9x2 inch baking pan."", ""Melt 1 tablespoon butter in medium nonstick skillet over medium-low heat. Add onion and saute until tender, about 10 minutes. Cool."", … ""Bake cornbread until golden and tester inserted comes out clean, about 45 minutes. Cool 20 minutes in pan. Cut cornbread into squares.""]","[""""]"
"""AllRecipes-59773cd5496dc7446c4…","""English""","""6663""","""Crispy Cheese Twists""","""AllRecipes""","""http://allrecipes.com/Recipe/6…","[""1/2 cup Parmesan cheese"", ""3/4 teaspoon ground black pepper"", … ""1 egg white""]","""http://images.media-allrecipes…","""These are great as an appetize…","[""Combine parmesan cheese, pepper and garlic powder. Unfold pastry sheets onto cutting board. Brush lightly with egg white; sprinkle each sheet with 1/4 of the cheese mixture. Lightly press into pastry, turn over; repeat. Cut each sheet into 12 (1-inch) strips; twist."", ""Place on ungreased cookie sheet and bake in 350 degrees F (175 degrees C) oven for 15 minutes or until golden brown.""]","[""""]"
"""AllRecipes-4022751697d6c412320…","""English""","""6665""","""Mom's Yeast Rolls""","""AllRecipes""","""http://allrecipes.com/Recipe/6…","[""2 cups hot water"", ""1/2 cup margarine"", … ""2 eggs""]","""http://images.media-allrecipes…","""This is the best bread recipe.…","[""Melt margarine in hot water. Add sugar and salt and stir. Add cold water and yeast. Stir to dissolve yeast."", ""Add 3 cups flour and mix. Add eggs and 2 1/2 - 3 cups more flour. Mix, cover and let rise until dough doubles in size."", … ""Make walnut size balls of dough. Place about 2 inches apart in well-buttered 9 x 13 inch pan. Bake in a preheated 350 degrees F (175 degrees C) oven for 30-45 minutes. Brush top of rolls with margarine while hot.""]","[""""]"


In [ ]:
with pl.Config(fmt_str_lengths=1000, tbl_width_chars=1000):
    print(reduced_ar_df.sample(100)['title','url'])

shape: (100, 2)
┌────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ title                                                                  ┆ url                                                                                                       │
│ ---                                                                    ┆ ---                                                                                                       │
│ str                                                                    ┆ str                                                                                                       │
╞════════════════════════════════════════════════════════════════════════╪═══════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ Dad's Hamburger Gravy                                              

AllRecipes data cleaning:
Need to give a different header/user-agent

```headers = {'
User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:50.0) Gecko/20100101 Firefox/50.0'}
```

1. Url checking: if response is not 200 status, drop recipe/add flag to drop recipe
2. If 200, 
    a. Get response.xpath('//*[@id="mntl-universal-breadcrumbs_1-0"]/li/a/span/text()').getall()
    b. Check if "Global Cuisines" in that array/list, if so, keep
    
    Global Cuisines, if so, add all the following tags as cuisines

    However, American cuisine may not count at AllRecipes

Otherwise, cuisine tag is ["Missing Cuisine"]

In [ ]:
# want to replace " " with "-"
# want to drop brackets and punctuation
# want to drop unicode

reduced_ar_df = reduced_ar_df.with_columns(
    pl.concat_str(
        [
            pl.col("url"),
            pl.col("title").str.normalize("NFKC")
                            .str.replace_all(r" ", "-")
                            .str.replace_all(r"[^a-zA-Z0-9-]", "")
                            .str.to_lowercase()
        ],
        separator="",
    ).alias("working_url")
)

with pl.Config(fmt_str_lengths=1000, tbl_width_chars=1000):
    print(reduced_ar_df)

shape: (225_602, 12)
┌─────────────────────────────────────────────────────────────────────────────┬──────────┬───────────┬────────────────────────────────────────────────────┬───┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
reduced_ar_df

mealeon_id,language,source_id,title,origin,url,ingredients,photo_url,description,steps,cuisines,working_url
str,str,str,str,str,str,list[str],str,str,list[str],list[str],str
"""AllRecipes-58c6dd19a6a89328e71…","""English""","""6664""","""Basil, Roasted Peppers and Mon…","""AllRecipes""","""http://allrecipes.com/Recipe/6…","[""1/2 cup unsalted butter, chilled and cubed"", ""1 cup chopped onion"", … ""1/2 cup chopped fresh basil""]","""http://images.media-allrecipes…","""I just started adding my favor…","[""Preheat oven to 400 degrees F (205 degrees C). Butter a 9x9x2 inch baking pan."", ""Melt 1 tablespoon butter in medium nonstick skillet over medium-low heat. Add onion and saute until tender, about 10 minutes. Cool."", … ""Bake cornbread until golden and tester inserted comes out clean, about 45 minutes. Cool 20 minutes in pan. Cut cornbread into squares.""]","[""""]","""http://allrecipes.com/Recipe/6…"
"""AllRecipes-59773cd5496dc7446c4…","""English""","""6663""","""Crispy Cheese Twists""","""AllRecipes""","""http://allrecipes.com/Recipe/6…","[""1/2 cup Parmesan cheese"", ""3/4 teaspoon ground black pepper"", … ""1 egg white""]","""http://images.media-allrecipes…","""These are great as an appetize…","[""Combine parmesan cheese, pepper and garlic powder. Unfold pastry sheets onto cutting board. Brush lightly with egg white; sprinkle each sheet with 1/4 of the cheese mixture. Lightly press into pastry, turn over; repeat. Cut each sheet into 12 (1-inch) strips; twist."", ""Place on ungreased cookie sheet and bake in 350 degrees F (175 degrees C) oven for 15 minutes or until golden brown.""]","[""""]","""http://allrecipes.com/Recipe/6…"
"""AllRecipes-4022751697d6c412320…","""English""","""6665""","""Mom's Yeast Rolls""","""AllRecipes""","""http://allrecipes.com/Recipe/6…","[""2 cups hot water"", ""1/2 cup margarine"", … ""2 eggs""]","""http://images.media-allrecipes…","""This is the best bread recipe.…","[""Melt margarine in hot water. Add sugar and salt and stir. Add cold water and yeast. Stir to dissolve yeast."", ""Add 3 cups flour and mix. Add eggs and 2 1/2 - 3 cups more flour. Mix, cover and let rise until dough doubles in size."", … ""Make walnut size balls of dough. Place about 2 inches apart in well-buttered 9 x 13 inch pan. Bake in a preheated 350 degrees F (175 degrees C) oven for 30-45 minutes. Brush top of rolls with margarine while hot.""]","[""""]","""http://allrecipes.com/Recipe/6…"
"""AllRecipes-a40c5e954eb4d5c570a…","""English""","""6666""","""Sweet Potato Bread I""","""AllRecipes""","""http://allrecipes.com/Recipe/6…","[""1 1/2 cups white sugar"", ""1/2 cup vegetable oil"", … ""1/2 cup chopped pecans""]","""http://images.media-allrecipes…","""A Southern delight with a 'mor…","[""Combine sugar and oil; beat well. Add eggs and beat. Combine flour, baking soda, salt, cinnamon and nutmeg. Stir flour mixture into egg mixture alternately with water. Stir in sweet potatoes and chopped nuts."", ""Pour batter into greased 9x5 inch loaf pan (or 2 small loaf pans). Bake at 350 degrees F (175 degrees C) for about one hour.""]","[""""]","""http://allrecipes.com/Recipe/6…"
"""AllRecipes-2d0452f4416810fc8b0…","""English""","""6668""","""Orange Buns""","""AllRecipes""","""http://allrecipes.com/Recipe/6…","[""1/4 cup butter"", ""1 teaspoon white sugar"", … ""1 teaspoon salt""]","""http://images.media-allrecipes…","""You can make these into braids…","[""Stir butter and 1 teaspoon sugar into the hot milk until butter is melted. When mixture is lukewarm, stir in yeast and set aside for 5 minutes."", ""When mixture is creamy, transfer to a large mixing bowl. Mix in 2 cups of bread flour. Add 1/2 cup sugar, eggs, orange juice, orange zest, and salt and beat until combined. Add remaining flour, mixing well after each addition, until it pulls away from the sides of the bowl."", … ""Bake until rolls or loaves are golden brown, 10 to 12 minutes for individual buns, about 20 minutes for braids. Frost rolls, if desired (see Cook's Note) o

In [ ]:
with pl.Config(fmt_str_lengths=1000, tbl_width_chars=1000):
    print(reduced_ar_df["working_url"].head(10))

shape: (10,)
Series: 'working_url' [str]
[
	"http://allrecipes.com/Recipe/6664/basil-roasted-peppers-and-monterey-jack-cornbreadbasil-roasted-peppers-and-monterey-jack-cornbread"
	"http://allrecipes.com/Recipe/6663/crispy-cheese-twistscrispy-cheese-twists"
	"http://allrecipes.com/Recipe/6665/moms-yeast-rollsmoms-yeast-rolls"
	"http://allrecipes.com/Recipe/6666/sweet-potato-bread-isweet-potato-bread-i"
	"http://allrecipes.com/Recipe/6668/orange-bunsorange-buns"
	"http://allrecipes.com/Recipe/6667/cornish-splitscornish-splits"
	"http://allrecipes.com/Recipe/6669/jalapeno-cheese-breadjalapeno-cheese-bread"
	"http://allrecipes.com/Recipe/6670/dees-health-breaddees-health-bread"
	"http://allrecipes.com/Recipe/6671/oatmeal-bread-ioatmeal-bread-i"
	"http://allrecipes.com/Recipe/6672/strawberry-bread-istrawberry-bread-i"
]


In [ ]:
reduced_ar_df = reduced_ar_df.drop("url").with_columns(
    pl.col("working_url").alias("url")
).drop("working_url")

reduced_ar_df.head(3)

mealeon_id,language,source_id,title,origin,ingredients,photo_url,description,steps,cuisines,url
str,str,str,str,str,list[str],str,str,list[str],list[str],str
"""AllRecipes-58c6dd19a6a89328e71…","""English""","""6664""","""Basil, Roasted Peppers and Mon…","""AllRecipes""","[""1/2 cup unsalted butter, chilled and cubed"", ""1 cup chopped onion"", … ""1/2 cup chopped fresh basil""]","""http://images.media-allrecipes…","""I just started adding my favor…","[""Preheat oven to 400 degrees F (205 degrees C). Butter a 9x9x2 inch baking pan."", ""Melt 1 tablespoon butter in medium nonstick skillet over medium-low heat. Add onion and saute until tender, about 10 minutes. Cool."", … ""Bake cornbread until golden and tester inserted comes out clean, about 45 minutes. Cool 20 minutes in pan. Cut cornbread into squares.""]","[""""]","""http://allrecipes.com/Recipe/6…"
"""AllRecipes-59773cd5496dc7446c4…","""English""","""6663""","""Crispy Cheese Twists""","""AllRecipes""","[""1/2 cup Parmesan cheese"", ""3/4 teaspoon ground black pepper"", … ""1 egg white""]","""http://images.media-allrecipes…","""These are great as an appetize…","[""Combine parmesan cheese, pepper and garlic powder. Unfold pastry sheets onto cutting board. Brush lightly with egg white; sprinkle each sheet with 1/4 of the cheese mixture. Lightly press into pastry, turn over; repeat. Cut each sheet into 12 (1-inch) strips; twist."", ""Place on ungreased cookie sheet and bake in 350 degrees F (175 degrees C) oven for 15 minutes or until golden brown.""]","[""""]","""http://allrecipes.com/Recipe/6…"
"""AllRecipes-4022751697d6c412320…","""English""","""6665""","""Mom's Yeast Rolls""","""AllRecipes""","[""2 cups hot water"", ""1/2 cup margarine"", … ""2 eggs""]","""http://images.media-allrecipes…","""This is the best bread recipe.…","[""Melt margarine in hot water. Add sugar and salt and stir. Add cold water and yeast. Stir to dissolve yeast."", ""Add 3 cups flour and mix. Add eggs and 2 1/2 - 3 cups more flour. Mix, cover and let rise until dough doubles in size."", … ""Make walnut size balls of dough. Place about 2 inches apart in well-buttered 9 x 13 inch pan. Bake in a preheated 350 degrees F (175 degrees C) oven for 30-45 minutes. Brush top of rolls with margarine while hot.""]","[""""]","""http://allrecipes.com/Recipe/6…"


In [ ]:
with pl.Config(fmt_str_lengths=1000, tbl_width_chars=1000):
    print(reduced_ar_df["url"].head(10))

shape: (10,)
Series: 'url' [str]
[
	"http://allrecipes.com/Recipe/6664/basil-roasted-peppers-and-monterey-jack-cornbreadbasil-roasted-peppers-and-monterey-jack-cornbread"
	"http://allrecipes.com/Recipe/6663/crispy-cheese-twistscrispy-cheese-twists"
	"http://allrecipes.com/Recipe/6665/moms-yeast-rollsmoms-yeast-rolls"
	"http://allrecipes.com/Recipe/6666/sweet-potato-bread-isweet-potato-bread-i"
	"http://allrecipes.com/Recipe/6668/orange-bunsorange-buns"
	"http://allrecipes.com/Recipe/6667/cornish-splitscornish-splits"
	"http://allrecipes.com/Recipe/6669/jalapeno-cheese-breadjalapeno-cheese-bread"
	"http://allrecipes.com/Recipe/6670/dees-health-breaddees-health-bread"
	"http://allrecipes.com/Recipe/6671/oatmeal-bread-ioatmeal-bread-i"
	"http://allrecipes.com/Recipe/6672/strawberry-bread-istrawberry-bread-i"
]


In [ ]:
from urllib.parse import urlparse

urlparse("http://allrecipes.com/Recipe/6672/")

ParseResult(scheme='http', netloc='allrecipes.com', path='/Recipe/6672/', params='', query='', fragment='')

In [ ]:
urlparse("http://allrecipes.com/Recipe/6672/").path

'/Recipe/6672/'

In [ ]:
urlparse("http://allrecipes.com/Recipe/6672/").path[8:-1]

'6672'

In [ ]:
urlparse("http://allrecipes.com/Recipe/6672/").path.split('/')[-2]

'6672'

In [ ]:
bbc_recipes_json_path = "../data/recipes-en-201706/bbccouk-recipes.json"

bbc_df = pl.read_ndjson(bbc_recipes_json_path)
bbc_df.head(3)

chef,chef_id,cooking_time_minutes,description,error,ingredients,instructions,instructions_detailed,photo_url,preparation_time_minutes,program,program_id,serves,time_scraped,title,total_time_minutes,url
str,str,i64,str,bool,list[str],list[str],list[struct[2]],str,i64,str,str,i64,i64,str,i64,str
"""James Tanner""","""james_tanner""",10,"""""",false,"[""250g/8¾oz plain flour, plus extra for dusting"", ""8 tbsp olive oil"", … ""1 free-range egg""]","[""Preheat the oven to 200C/400F/Gas 6."", ""For the pizza base, place the flour, oil, water and salt into a food processor and blend together until a dough is formed. Tip out onto a floured work surface and knead. Shape into a round base about 20cm/8in wide."", … ""Serve on a large plate, and slice into wedges to serve.""]","[{""plain flour"",""250g/8¾oz plain flour, plus extra for dusting""}, {""olive oil"",""8 tbsp olive oil""}, … {""egg"",""1 free-range egg""}]",null,30,"""Ready Steady Cook""","""b006vcgr""",1,1499227761,"""Ten-minute pizza""",40,"""http://bbc.co.uk/food/recipes/…"
"""Mary Berry""","""mary_berry""",0,"""This is my standby pasta suppe…",false,"[""350g/12oz penne pasta"", ""2 x 80g/3oz packs Parma ham, snipped into small pieces"", … ""crunchy bread""]","[""Cook the pasta in a pan of boiling salted water according to the packet instructions. Drain and set aside"", ""Heat a frying pan until hot. Add the pieces of Parma ham and fry until crisp, remove half of the ham onto a plate and set aside. Add the mushrooms to the pan and fry for two minutes. Add the crème fraîche and bring up to the boil. Add the pasta, Parmesan and parsley and toss together over the heat. Season well with salt and pepper."", ""Serve with a green salad and crunchy bread.""]","[{""pasta"",""350g/12oz penne pasta""}, {""ham"",""2 x 80g/3oz packs Parma ham, snipped into small pieces""}, … {""bread"",""crunchy bread""}]","""http://ichef.bbci.co.uk/food/i…",30,"""Mary Berry Cooks""","""p01s4q10""",6,1499227763,"""15 minute pasta""",30,"""http://bbc.co.uk/food/recipes/…"
"""Mari Williams""","""mari_williams""",0,"""Simple 3D iced biscuits inspir…",false,"[""200g/7oz unsalted soft butter"", ""200g/7oz caster sugar"", … ""ready-to-roll icing, in colours of your choice, to decorate the biscuits""]","[""To make the basic dough, line a baking tray with baking parchment (you may need two baking trays or, alternatively, cook the biscuits in batches)."", ""In a bowl, use a wooden spoon or a mixer to cream together the butter, sugar and vanilla essence until just creamy. Do not over work the batter at this stage. Beat in the egg until well combined."", … ""To decorate the sitting down person biscuits, decorate the biscuit bodies as desired. Slot the legs sideways into the precut slots so the person is sitting down with their legs stretched out in front of them. Attach the feet to the bottom of the legs using icing and leave to dry.""]","[{""butter"",""200g/7oz unsalted soft butter""}, {""caster sugar"",""200g/7oz caster sugar""}, … {null,""ready-to-roll icing, in colours of your choice, to decorate the biscuits""}]","""http://ichef.bbci.co.uk/food/i…",30,"""The Great British Bake Off""","""b013pqnm""",0,1499227766,"""3D biscuits """,30,"""http://bbc.co.uk/food/recipes/…"


In [ ]:
# ar_df = ar_df.with_columns(
#     pl.concat_str(
#         [
#             pl.col("url"),
#             pl.col("title").str.normalize("NFKC")
#                             .str.replace_all(r" ", "-")
#                             .str.replace_all(r"[^a-zA-Z0-9-]", "")
#                             .str.to_lowercase()
#         ],
#         separator="",
#     ).alias("working_url"),
#     (pl.col("url").map_elements(partial(
#                                     text_processing.mealeon_id_assigner, 
#                                     "AllRecipes"
#                                 )
#                                 , return_dtype=pl.String)
#                     .alias("mealeon_id")),
#     (pl.col("instructions").alias("steps")),
#     (pl.col("url").map_elements(text_processing.allrecipes_id_extractor, return_dtype=pl.String).alias("source_id")),
#     (pl.lit("English").alias("language")),
#     (pl.lit("AllRecipes").alias("origin")),
#     (pl.lit([""]).alias("cuisines")),
# ).drop(
#     ["url"]
# ).rename({"working_url":"url"})

# ar_df.head(3)
with pl.Config(fmt_str_lengths=1000, tbl_width_chars=1000):
    print(bbc_df["url"].head())

shape: (10,)
Series: 'url' [str]
[
	"http://bbc.co.uk/food/recipes/10minutepizza_87314"
	"http://bbc.co.uk/food/recipes/15_minute_pasta_33407"
	"http://bbc.co.uk/food/recipes/3d_biscuits_29555"
	"http://bbc.co.uk/food/recipes/2_hour_christmas_dinner_79341"
	"http://bbc.co.uk/food/recipes/_81487"
	"http://bbc.co.uk/food/recipes/aberdeenbutteriesrow_92370"
	"http://bbc.co.uk/food/recipes/abundant_steamed_sea_22932"
	"http://bbc.co.uk/food/recipes/aclassicspongecakewi_9406"
	"http://bbc.co.uk/food/recipes/3wayswithlemoncurd_67266"
	"http://bbc.co.uk/food/recipes/acreamycrunchyfruits_92490"
]


In [ ]:
# bbc_df = bbc_df.with_columns(
#     (pl.col("url").map_elements(text_processing.mealeon_id_bbc_assigner, return_dtype=pl.String).alias("mealeon_id")),
#     (pl.col("instructions").alias("steps")),
#     (pl.col("url").alias("source_id")),
#     (pl.lit("English").alias("language")),
#     (pl.lit("BBC").alias("origin")),
#     (pl.lit(["Missing Cuisine"]).alias("cuisines")),
# )

# reduced_bbc_df = bbc_df.select(["mealeon_id", "language", "source_id", "title", "origin", "url", "ingredients", "photo_url", "description", "steps", "cuisines"])
# reduced_bbc_df.head(3)

BBC might be better to just create a new scraper: there are cuisine filters and pagination

In [ ]:
cookstr_recipes_json_path = "../data/recipes-en-201706/cookstr-recipes.json"

cookstr_df = pl.read_ndjson(cookstr_recipes_json_path)
cookstr_df.head(3)

ingredients,makes,difficulty,occasion,photo_url,url,kid_friendly,cookbook_publisher,comment_count,cost,course,photo_credit_site,make_ahead,date_modified,cooking_method,dietary_considerations,error,total_time,copyright,time_scraped,contributors,photo_credit_name,ingredients_detailed,rating_count,taste_and_texture,rating_value,description,type_of_dish,chef,instructions,meal,title,cookbook
list[str],str,str,str,str,str,bool,str,i64,str,str,str,bool,str,str,str,bool,str,str,i64,list[str],str,list[struct[2]],i64,str,i64,str,str,str,list[str],str,str,str
"[""1 tablespoon softened butter"", ""2 tablespoons flour"", … ""1 tablespoon finely grated orange rind""]","""1 3-layer cake""",null,"""Buffet, Casual Dinner Party, F…","""https://irepo.primecp.com/2016…","""https://www.cookstr.com/recipe…",true,"""Running Press""",0,"""Inexpensive""","""dessert""","""http://www.jdeleophoto.com""",true,"""2017-06-01""","""baking""","""peanut free, soy free, tree nu…",false,"""under 2 hours""","""1977 James Beard""",1499058605,null,"""Joseph DeLeo""","[{[""butter""],""1 tablespoon softened butter""}, {[""flour""],""2 tablespoons flour""}, … {[],""1 tablespoon finely grated orange rind""}]",0,"""buttery, fruity, sweet""",null,"""This layer cake was a standard…","""cake""","""James Beard""","[""Preheat the oven to 350°. Butter the bottom and sides of each layer-cake pan with the softened butter, using your hands, then sprinkle the flour inside and shake this around so you get a thin coating on the butter. Tip out any excess."", ""Now to sift your flour. Lay a large piece of waxed paper on a board, put a dry measuring cup in the center, hold a sifter directly over it, scoop cake flour from the package into the sifter, and sift the flour directly into the cup. When the cup is full, draw the back of a knife blade lightly across the top of the cup (don’t shake the flour down, or it will become dense) and then tip the measured flour into a mixing bowl. Repeat with the other 2 cups of flour (you can put any flour that spilled onto the waxed paper back in the sifter)."", … ""Leave the cake to cool.""]","""dinner, snack, tea""","""1-2-3-4 Cake""","""James Beard's Theory and Pract…"
"[""2 garlic cloves, finely minced"", ""4 scallions, thinly sliced"", … ""Kosher salt and freshly ground black pepper""]",null,null,"""Casual Dinner Party, Formal Di…","""https://irepo.primecp.com/2016…","""https://www.cookstr.com/recipe…",true,"""Ecco""",0,"""Splurge""","""Main Course""",null,null,"""2016-11-16""","""Grilling""",null,false,"""under 4 hours""","""2008 Mario Batali""",1499072043,null,null,"[{[""garlic""],""2 garlic cloves, finely minced""}, {[""scallions""],""4 scallions, thinly sliced""}, … {[""Kosher salt""],""Kosher salt and freshly ground black pepper""}]",0,"""Cheesy, Meaty, Savory""",null,"""Beef braciole is traditionally…",null,"""Mario Batali""","[""In a medium bowl, combine the garlic, scallions, parsley, salami, Fontina, Parmigiano, and bread crumbs and mix well. Add ¼ cup of the olive oil and mix well with your hands or a spoon. Set aside."", ""Cut six 15-inch-long pieces of kitchen twine. Open out the beef, season on both sides with salt and pepper, and place it on a work surface so a long side is toward you. Spread the bread crumb mixture evenly over the beef, leaving a ½-inch border along the side farthest from you; press and gently pack the stuffing mixture onto the beef to keep it in place (you may have a little stuffing left over–it makes a great panini filling). Starting from the side nearest you, roll up the meat like a jelly roll, pressing any stuffing that falls out of the ends back into the roll, and tie tightly with the twine, spacing the ties evenly (it’s easier if you have a friend to tie the beef while you hold the roll together). Wrap tightly in plastic wrap to make a compact roll, and place in the refrigerator for at least 2 hours, or as long as overnight."", … ""Gently lay the pinwheels on the hottest part of the grill and cook, unmoved,

In [ ]:
with pl.Config(fmt_str_lengths=1000, tbl_width_chars=1000):
    print(cookstr_df["url"].head())

shape: (10,)
Series: 'url' [str]
[
	"https://www.cookstr.com/recipes/1-2-3-4-cake-james-beard"
	"https://www.cookstr.com/recipes/Beef-Braciole-Pinwheel-Style"
	"https://www.cookstr.com/recipes/Bombay-Eggs"
	"https://www.cookstr.com/recipes/Chipotle-Meatballs"
	"https://www.cookstr.com/recipes/Chocolate-Chestnut-Buche-de-Noel"
	"https://www.cookstr.com/recipes/Classic-Brioche"
	"https://www.cookstr.com/recipes/Conch-Salad-Bahamian-Style"
	"https://www.cookstr.com/recipes/Grilled-Corn-with-Chipotle-Butter-and-Parmesean-Cheese"
	"https://www.cookstr.com/recipes/Grilled-Peaches-with-Balsamic-and-Pepper-Sauce"
	"https://www.cookstr.com/recipes/Herb-and-Spice-Roasted-Chicken"
]


In [ ]:
cookstr_df = cookstr_df.with_columns(
    (pl.col("url").map_elements(partial(
                                    text_processing.mealeon_id_assigner, 
                                    "Cookstr"
                                )
                                , return_dtype=pl.String)
                    .alias("mealeon_id")),
    (pl.col("url").alias("source_id")),
    (pl.col("instructions").alias("steps")),
    (pl.lit("English").alias("language")),
    (pl.lit("Cookstr").alias("origin")),
    (pl.lit(["Missing Cuisine"]).alias("cuisines")),
)

reduced_cookstr_df = cookstr_df.select(["mealeon_id", "language", "source_id", "title", "origin", "url", "ingredients", "photo_url", "description", "steps", "cuisines"])
reduced_cookstr_df.head(3)

mealeon_id,language,source_id,title,origin,url,ingredients,photo_url,description,steps,cuisines
str,str,str,str,str,str,list[str],str,str,list[str],list[str]
"""Cookstr-c6bc49ba4b02533264a410…","""English""","""https://www.cookstr.com/recipe…","""1-2-3-4 Cake""","""Cookstr""","""https://www.cookstr.com/recipe…","[""1 tablespoon softened butter"", ""2 tablespoons flour"", … ""1 tablespoon finely grated orange rind""]","""https://irepo.primecp.com/2016…","""This layer cake was a standard…","[""Preheat the oven to 350°. Butter the bottom and sides of each layer-cake pan with the softened butter, using your hands, then sprinkle the flour inside and shake this around so you get a thin coating on the butter. Tip out any excess."", ""Now to sift your flour. Lay a large piece of waxed paper on a board, put a dry measuring cup in the center, hold a sifter directly over it, scoop cake flour from the package into the sifter, and sift the flour directly into the cup. When the cup is full, draw the back of a knife blade lightly across the top of the cup (don’t shake the flour down, or it will become dense) and then tip the measured flour into a mixing bowl. Repeat with the other 2 cups of flour (you can put any flour that spilled onto the waxed paper back in the sifter)."", … ""Leave the cake to cool.""]","[""Missing Cuisine""]"
"""Cookstr-35b822211e2337e3bdadfe…","""English""","""https://www.cookstr.com/recipe…","""Beef Braciole ""Pinwheel-Style""""","""Cookstr""","""https://www.cookstr.com/recipe…","[""2 garlic cloves, finely minced"", ""4 scallions, thinly sliced"", … ""Kosher salt and freshly ground black pepper""]","""https://irepo.primecp.com/2016…","""Beef braciole is traditionally…","[""In a medium bowl, combine the garlic, scallions, parsley, salami, Fontina, Parmigiano, and bread crumbs and mix well. Add ¼ cup of the olive oil and mix well with your hands or a spoon. Set aside."", ""Cut six 15-inch-long pieces of kitchen twine. Open out the beef, season on both sides with salt and pepper, and place it on a work surface so a long side is toward you. Spread the bread crumb mixture evenly over the beef, leaving a ½-inch border along the side farthest from you; press and gently pack the stuffing mixture onto the beef to keep it in place (you may have a little stuffing left over–it makes a great panini filling). Starting from the side nearest you, roll up the meat like a jelly roll, pressing any stuffing that falls out of the ends back into the roll, and tie tightly with the twine, spacing the ties evenly (it’s easier if you have a friend to tie the beef while you hold the roll together). Wrap tightly in plastic wrap to make a compact roll, and place in the refrigerator for at least 2 hours, or as long as overnight."", … ""Gently lay the pinwheels on the hottest part of the grill and cook, unmoved, for 5 to 7 minutes. Using a spatula, carefully turn each pinwheel over and cook for about 4 minutes longer for medium-rare. (Don’t be alarmed if some of the cheese in the stuffing starts to melt and char on the grill, making kind of a savory Florentine-cookie-like thing; but if you find it charring too much, move the pinwheels to a slightly cooler part of the grill.) Transfer to a platter and serve.""]","[""Missing Cuisine""]"
"""Cookstr-7002a65f0fccf3dd80470f…","""English""","""https://www.cookstr.com/recipe…","""Bombay Eggs""","""Cookstr""","""https://www.cookstr.com/recipe…","[""1 teaspoon cumin seeds"", ""2 teaspoons coriander seeds"", … ""1 cup homemade or Greek yogurt""]","""https://irepo.primecp.com/2016…","""Editor's Note: Enjoy a very, v…","[""In a large, lidded saucepan, toast the cumin and coriander seeds over a medium to high heat, swirling the pan around until the spices are a pale golden brown. It should take 2 to 3 minutes. Then lightly grind them in a mortar and pestle."", ""Pour the oil into the pan on a medium heat and, when it's hot, put the spices back into the pan. Stir-fry for a minute, then add the onions. Cook for 6 

Cookstr appears to be down

In [ ]:
# let's reinvestigate epicurious data
epic_df = pl.read_parquet("../data/processed/cleaned_df.parquet.gzip")

epic_df.head(3)

dek,hed,aggregateRating,ingredients,prepSteps,reviewsCount,willMakeAgainPct,ingredients_lemmafied,cuisine_name,photo_filename,photo_credit,author_name,date_published,recipe_url,id
str,str,f64,list[str],list[str],i64,i64,str,str,str,str,str,"datetime[ns, UTC]",str,str
"""How does fried chicken achieve…","""Pickle-Brined Fried Chicken""",3.11,"[""1 tablespoons yellow mustard seeds"", ""1 tablespoons brown mustard seeds"", … ""A deep-fry thermometer""]","[""Toast mustard and coriander seeds in a dry medium saucepan over medium heat, tossing often, until mustard seeds begin to pop, about 3 minutes. Add vinegar, salt, and sugar and bring to a boil. Reduce heat and simmer, stirring often, until salt and sugar are dissolved, about 4 minutes. Remove from heat; stir in dill and 4 cups water. Let cool."", ""Place chicken and brine in a large resealable plastic bag; chill 3 hours. Remove chicken from brine, scraping off seeds, cover, and chill until ready to fry."", … ""Drizzle chicken with honey; sprinkle with sea salt and benne seeds. Serve with hot sauce alongside.""]",7,100,"""tablespoon yellow mustard seed…","""Missing Cuisine""","""51247610_fried-chicken_1x1.jpg""","""Michael Graydon and Nikole Her…","""Missing Author Name""",2014-08-19 04:00:00 UTC,"""https://www.epicurious.com/rec…","""54a2b6b019925f464b373351"""
"""Spinaci all'Ebraica""","""Spinach Jewish Style""",3.22,"[""3 pounds small-leaved bulk spinach"", ""Salt"", … ""Dash nutmeg""]","[""Remove the stems and roots from the spinach. Rinse in many changes of cold water until any trace of sand is removed. Place in a large pot with a pinch of salt and no water other than that retained from washing. Cook over moderately high heat, covered, for 5 minutes. Drain."", ""Soak the raisins in the lukewarm water for a couple of minutes, then drain."", ""Meanwhile, heat the oil in a large skillet, add the onion and sauté until the onion is soft and translucent; add the raisins, pignoli, and small amounts of salt and pepper. Sauté, stirring, 1 minute. Add the spinach and nutmeg and sauté, stirring frequently, until the spinach looks dry and crisp.""]",5,80,"""pound small leave bulk spinach…","""Italian""","""EP_12162015_placeholders_rusti…","""Photo by Chelsea Kyle, Prop St…","""Edda Servi Machlin""",2008-09-09 04:00:00 UTC,"""https://www.epicurious.com/rec…","""54a408a019925f464b3733bc"""
"""This majestic, moist, and rich…","""New Year’s Honey Cake""",3.62,"[""3 1/2 cups all-purpose flour"", ""1 tablespoon baking powder"", … ""1/2 cup slivered or sliced almonds (optional)""]","[""I like this cake best baked in a 9-inch angel food cake pan, but you can also make it in a 10-inch tube or bundt cake pan, a 9 by 13-inch sheetpan, or three 8 by 4 1/2-inch loaf pans."", ""Preheat the oven to 350°F. Lightly grease the pan(s). For tube and angel food pans, line the bottom with lightly greased parchment paper. For gift honey cakes, I use ""cake collars"" (available from Sweet Celebrations) designed to fit a specific loaf pan. These give the cakes an appealing, professional look."", … ""Let the cake stand for 15 minutes before removing it from the pan. Then invert it onto a wire rack to cool completely.""]",105,88,"""cup purpose flour brk tablespo…","""Kosher""","""EP_09022015_honeycake-2.jpg""","""Photo by Chelsea Kyle, Food St…","""Marcy Goldman""",2008-09-10 04:00:00 UTC,"""https://www.epicurious.com/rec…","""54a408a26529d92b2c003631"""


In [ ]:
reduced_epic_df = epic_df.with_columns(
    (pl.col("recipe_url").map_elements(partial(
                                    text_processing.mealeon_id_assigner, 
                                    "Epicurious"
                                )
                                , return_dtype=pl.String)
                    .alias("mealeon_id")),
    (pl.lit("English").alias("language")),
    (pl.lit("Epicurious").alias("origin")),
    (pl.concat_arr("cuisine_name").alias("cuisines"))
).rename(
    {
        "dek": "description",
        "hed":"title",
        "recipe_url": "url",
        "id": "source_id",
        "prepSteps":"steps",
        # "cuisine_name": "cuisines"
    }
)

reduced_epic_df = reduced_epic_df.drop(["reviewsCount", "willMakeAgainPct", "ingredients_lemmafied", "photo_filename", "photo_credit", "author_name", "date_published", "cuisine_name", "aggregateRating"])

reduced_epic_df.head(1)

description,title,ingredients,steps,url,source_id,mealeon_id,language,origin,cuisines
str,str,list[str],list[str],str,str,str,str,str,"array[str, 1]"
"""How does fried chicken achieve…","""Pickle-Brined Fried Chicken""","[""1 tablespoons yellow mustard seeds"", ""1 tablespoons brown mustard seeds"", … ""A deep-fry thermometer""]","[""Toast mustard and coriander seeds in a dry medium saucepan over medium heat, tossing often, until mustard seeds begin to pop, about 3 minutes. Add vinegar, salt, and sugar and bring to a boil. Reduce heat and simmer, stirring often, until salt and sugar are dissolved, about 4 minutes. Remove from heat; stir in dill and 4 cups water. Let cool."", ""Place chicken and brine in a large resealable plastic bag; chill 3 hours. Remove chicken from brine, scraping off seeds, cover, and chill until ready to fry."", … ""Drizzle chicken with honey; sprinkle with sea salt and benne seeds. Serve with hot sauce alongside.""]","""https://www.epicurious.com/rec…","""54a2b6b019925f464b373351""","""Epicurious-7585472a912c3cde742…","""English""","""Epicurious""","[""Missing Cuisine""]"


#!/usr/bin/env python3

import json
import os
import sys
import unicodedata

data_dir = sys.argv[1]
doc_type = sys.argv[2]
fields = sys.argv[3].split(",")

docs_file = os.path.join(data_dir, "epicurious-recipes_m2.json")
out_file = os.path.join(data_dir, "mealeon_vespa.json")


def fields_populator(k, v):
    lst_keys = ["ingredients", "prepSteps"]

    if k in lst_keys:
        field = (
            "".join(
                cha if unicodedata.category(cha)[0] != "C" else " "
                for sen in v
                for cha in sen + "|"
            )
            .rstrip("|")
            .split("|")
        )

    elif k == "tag":
        field = [
            v["name"] if v["name"] and v["category"] == "cuisine" else "Missing Cuisine"
        ]

    elif k == "id":
        field = f"epicurious-{v}"

    else:
        field = v

    return field


def main():
    renamer_dict = {
        "id": "id",
        "hed": "title",
        "ingredients": "ingredients",
        "prepSteps": "steps",
        "tag": "cuisine",
    }

    with open(docs_file) as f, open(out_file, "w") as out:
        d = json.load(f)

        output_vespa = []
        for record in d:
            vespa_record = {
                "put": f"id:{doc_type}:{doc_type}::epicurious-{record['id']}",
                "fields": {
                    "origin": "epicurious",
                    "id": "",
                    "title": "",
                    "ingredients": "",
                    "steps": "",
                    "cuisine": "",
                },
            }

            vespa_record["fields"].update(
                {
                    renamer_dict[k]: fields_populator(k, v)
                    for k, v in record.items()
                    if k in renamer_dict.keys()
                }
            )

            output_vespa.append(vespa_record)

        json.dump(output_vespa, out)


if __name__ == "__main__":
    main()


In [ ]:
# make sure postgres is running!
# load postgres credentials
postgres_key_path = "../secrets/postgres_login.json"
with open(postgres_key_path, "r") as fo:
    postgres_key = json.loads(fo.read())
    user = postgres_key["user"]
    password = postgres_key["password"]
    host = postgres_key["host"]


In [ ]:
# try connecting to Postgresql with ADBC
uri = f"postgresql://{user}:{password}@{host}/mealeon"

# reduced_ar_df.write_database(table_name="recipe_scrapes", connection=uri, engine="adbc", if_table_exists="append")
# reduced_bbc_df.write_database(table_name="recipe_scrapes", connection=uri, engine="adbc", if_table_exists="append")
# reduced_cookstr_df.write_database(table_name="recipe_scrapes", connection=uri, engine="adbc", if_table_exists="append")
reduced_epic_df.write_database(table_name="recipe_scrapes", connection=uri, engine="adbc", if_table_exists="append")

34756

In [ ]:
all_recipes_json_path = "../data/raw/allrecipes_file_scrapes.jsonl"

ar_df = pl.read_ndjson(all_recipes_json_path)
ar_df.head(3)

source_id,mealeon_id,title,url,description,cuisines,ingredients,origin,language,photo_url,steps
str,str,str,str,str,list[str],list[str],str,str,str,list[str]
"""221067""","""AllRecipes-aae159ff27db03918d3…","""Creamy Ricotta Pie""","""https://www.allrecipes.com/rec…","""This is a creamy pie, not a de…","[""Missing Cuisine""]","[""2 cups all-purpose flour"", ""½ cup white sugar (Optional)"", … ""1 pinch white sugar, for sprinkling (Optional)""]","""AllRecipes""","""English""","""https://www.allrecipes.com/thm…","[""Preheat oven to 350 degrees F (175 degrees C)."", ""Mix flour, 1/2 cup sugar, shortening, 2 eggs, baking powder, 2 teaspoons vanilla extract, and salt together in a bowl until dough comes together."", … ""The pie is not firm when removed from oven but will firm up after chilling for a few hours.""]"
"""220733""","""AllRecipes-5e43d8a5104033272d1…","""Apple Caramel Bars""","""https://www.allrecipes.com/rec…","""We're pretty sure that this is…","[""Missing Cuisine""]","[""1 box Duncan Hines® Apple Caramel Cake Mix"", ""1 egg"", … ""1 cup chopped peanuts (half reserved for the top, half mixed with caramel pouch)""]","""AllRecipes""","""English""","""https://www.allrecipes.com/thm…","[""Preheat oven to 350 degrees F."", ""In small sauce pan, melt brown sugar, corn syrup and butter on medium heat. Bring to a boil and cook for 3 minutes. Remove from heat."", … ""Remove from oven and immediately press down on surface of hot cooked bars with spatula. Cool and refrigerate before cutting.""]"
"""235127""","""AllRecipes-8c96ce475b268a54e9e…","""Cheesy Zucchini Rice Bake""","""https://www.allrecipes.com/rec…","""A gooey, cheesy side dish that…","[""Missing Cuisine""]","[""1 egg"", ""1 cup milk"", … ""1 ½ cups shredded Cheddar cheese, or more to taste""]","""AllRecipes""","""English""","""https://www.allrecipes.com/thm…","[""Preheat oven to 375 degrees F (190 degrees C)."", ""Beat the egg in a large bowl. Pour milk and margarine into the egg. Slowly stream the soup into the egg mixture while stirring to prevent lumps. Stir rice, zucchini, and Cheddar cheese into the mixture; pour into an 8-inch baking dish."", ""Bake in preheated oven until set, about 30 minutes.""]"


In [ ]:
# make sure postgres is running!
# load postgres credentials
postgres_key_path = "../secrets/postgres_login.json"
with open(postgres_key_path, "r") as fo:
    postgres_key = json.loads(fo.read())
    user = postgres_key["user"]
    password = postgres_key["password"]
    host = postgres_key["host"]

    uri = f"postgresql://{user}:{password}@{host}/mealeon"

    ar_df.write_database(table_name="recipe_scrapes", connection=uri, engine="adbc", if_table_exists="append")
